# **CS 181 HW5**
---

## Problem 2

#### Initialize data and parameters

Consider a specific example of when we have $K = 3$ component Gamma distributions. Let's initialize the initial parameter values for $\theta$ and $\beta_k$ as follows:
$$
\begin{align*}
  \theta_k &=  1/K, \\
  \beta_k & = k/K.
\end{align*}
$$

Note that we usually initialize $\theta$ and $\beta_k$ randomly. However, by fixing the initial $\theta$ and $\beta_k$, EM becomes deterministic which makes debugging (and grading) easier.



In [36]:
import torch
import torch.distributions as ds
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

x = torch.load('data.pt')
if isinstance(x, np.ndarray):
    x = torch.from_numpy(x)


In [14]:
# # uncomment to use numpy (optional)
# import numpy as np
# from scipy.stats import gamma
# x = x.numpy()
# theta = theta.numpy()
# betas = betas.numpy()

#### **Todo:** implement the E-step

In [37]:
def e_step(theta, betas):
    q = ds.Gamma(alpha, betas).log_prob(x) + theta.log() 
    q = q - q.logsumexp(dim=1, keepdim=True)
    q = q.exp() 
    return q

#### **Todo:** implement the M-step

In [38]:
def m_step(q):
    theta_hat = q.mean(dim=0)
    betas_hat = (alpha * q.sum(dim=0)) / (q * x.unsqueeze(-1)).sum(dim=0)
    return theta_hat, betas_hat

#### **Todo:** implement log likelihood

In [43]:
def log_px(x, theta, betas):
    p = torch.zeros(x.shape[0], len(betas))
    for k in range(len(betas)):
        gamma_dist = ds.Gamma(alpha, betas[k])
        p[:, k] = theta[k] * gamma_dist.log_prob(x).exp()
    return torch.log(p.sum(dim=1))

def log_likelihood(theta, betas):
    return log_px(x, theta, betas).sum()

#### **Todo:** implement EM algorithm

In [44]:
def run_em(theta, betas, iterations=1000):
    for _ in range(iterations):
        q = e_step(theta, betas)
        theta, betas = m_step(q)
    return theta, betas

#### Plot 

In [45]:
def make_overlay_plot(theta, betas):
    x_test = torch.linspace(0.01, x.max(), 1000).unsqueeze(1)  # Make x_test two-dimensional
    prob = log_px(x_test, theta, betas).exp()
    # prob = np.exp(log_px(x_test.unsqueeze(-1), theta, betas))  # use this line for numpy
    ll = log_likelihood(theta, betas)
    
    fig, ax = plt.subplots(figsize=(5, 3))
    fig.subplots_adjust(top=0.7)
    fig.suptitle(f'theta = {theta}\nbeta = {betas}\nlog likelihood = {ll:.3e}')
    
    ax.hist(x.T, bins=100, color='tomato', alpha=0.5, density=True, label='Dataset')
    ax.plot(x_test, prob, color='royalblue', label='Gamma mixture')
    
    ax.set_title(f'Dataset and Gamma mixture (K={len(theta)})')
    ax.set_xlabel('Recovery time (hours)')
    ax.set_ylabel('Density')
    ax.legend()

In [46]:
alpha = 5.0
for K in range(1,5):
    theta0 = torch.ones(K) / K
    betas0 = (torch.arange(K) + 1) / K
    # theta0 = np.ones(K) / K               # use this for numpy
    # betas0 = (np.arange(K) + 1) / K
    
    theta, betas = run_em(theta0, betas0)
    
    make_overlay_plot(theta, betas)
    plt.savefig(f'p2_3_{K}mixtures.pdf', bbox_inches='tight')

RuntimeError: expand(torch.FloatTensor{[1000, 1]}, size=[1000]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (2)

---
## Problem 3

#### Initialize data:

In [47]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True)  # download MNIST
N = 6000 

x = mnist_trainset.data[:N]  # select N datapoints
x = x.flatten(1)             # flatten the images
x = x.float()                # convert pixels from uint8 to float
# x = x.numpy()              # uncomment to use numpy (optional)

100%|██████████| 9912422/9912422 [00:00<00:00, 13007689.73it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 92329034.93it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 11674282.07it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17955258.03it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



#### **Todo:** implement PCA

*Hint: see `.linalg.svd()`*

In [53]:
def pca(x, n_comps=500):
    x_centered = x - x.mean(dim=0)
    cov_matrix = torch.matmul(x_centered.T, x_centered) / (x.size(0) - 1)
    eigenvalues, eigenvectors = torch.linalg.eigh(cov_matrix, UPLO='U')
    top_eigvals = eigenvalues[-n_comps:].flip(dims=(0,))
    top_pcomps = eigenvectors[:, -n_comps:].flip(dims=(1,))
    return top_eigvals, top_pcomps

#### **Todo:** calculate cumulative fraction of variance

*Hint: see `.cumsum()`*

In [54]:
def calc_cfvs(eigvals):
    total_variance = torch.sum(eigvals)
    cfvs = torch.cumsum(eigvals, dim=0) / total_variance
    return cfvs

#### **Todo:** calculate mean squared L2 norm reconstruction loss

In [55]:
def calc_errs(x, pcomps):
    x_centered = x - x.mean(dim=0)
    x_projected = torch.matmul(x_centered, pcomps)
    reconstruction = torch.matmul(x_projected, pcomps.T) + x.mean(dim=0)
    err_mean = (x_centered ** 2).mean()
    err_pcomp = ((x - reconstruction) ** 2).mean()
    return err_mean, err_pcomp

#### Plot and print errors:

In [56]:
def plot_pic(pic, ax, title=''):
    x = pic.reshape(28, 28)
    ax.imshow(x, cmap='binary')
    ax.set_title(title)
    ax.axis('off')

def make_plots(eigvals, cfvs, x_mean, pcomps):
    # plot eigenvals and cfvs
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
    ax1.plot(eigvals, color='tomato')
    ax1.set_title('Eigenvalues')
    ax2.plot(cfvs, color='tomato')
    ax2.set_title('CFVs')
    fig.savefig('p3_cfvs.pdf')

    # plot mean
    fig, ax = plt.subplots(1, 1, figsize=(3, 3))
    plot_pic(x_mean, ax, title='Mean')
    fig.savefig('p3_mean.pdf')

    # plot top 10 pcomps
    fig, axes = plt.subplots(2, 5, figsize=(10, 4))
    for i in range(10):
        plot_pic(pcomps[i], axes.flat[i], title=f'PC index {i}')
    fig.savefig('p3_pcomps.pdf')

In [57]:
# do PCA
eigvals, pcomps = pca(x)

# calculate CFVs
fcvs = calc_cfvs(eigvals)

# print errors
err_mean, err_pcomp = calc_errs(x, pcomps)
print(f'Reconstruction error (using mean): {err_mean:3e}')
print(f'Reconstruction error (using mean and top 10 pcomps): {err_pcomp:3e}')

# make plots
make_plots(eigvals, fcvs, x.mean(0), pcomps)


RuntimeError: size mismatch, got 6000, 6000x784,500